In [250]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib
import matplotlib as plt
from matplotlib.pyplot import figure
from datetime import datetime
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,8)



In [251]:
df = pd.read_csv("DataSet/movies.csv")
df.head()

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000.0,58853106.0,Columbia Pictures,104.0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000.0,538375067.0,Lucasfilm,124.0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000.0,83453539.0,Paramount Pictures,88.0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000.0,39846344.0,Orion Pictures,98.0


## Data cleansing

In [252]:
#Check for any missing data 
missing_data_nr = len(df[df.isnull().any(axis=1)])
print(f"The number of the missing data on this dataset is: {missing_data_nr}")

#Checking mising values per each col
missing_per_col = df.isnull().sum() 
print(missing_per_col)

#Percentage of records missing for each column
for col in df.columns:
    pct_missing = np.mean(df[col].isnull())
#     print('{} - {}%'.format(col,pct_missing))


#I plan to use budget for correlation and budget has most missing values, so I will drop all records with missing values 
#2247 missing values, from them 2171 are on the budget column

df = df.dropna()

The number of the missing data on this dataset is: 2247
name           0
rating        77
genre          0
year           0
released       2
score          3
votes          3
director       0
writer         3
star           1
country        3
budget      2171
gross        189
company       17
runtime        4
dtype: int64


In [253]:
df.dtypes
#Change unnesecary floats into int
df['budget'] = df['budget'].astype('int64')
df['gross'] = df['gross'].astype('int64')
df['votes'] = df['votes'].astype('int64')
df['runtime'] = df['runtime'].astype('int64')

df.dtypes

name         object
rating       object
genre        object
year          int64
released     object
score       float64
votes         int64
director     object
writer       object
star         object
country      object
budget        int64
gross         int64
company      object
runtime       int64
dtype: object

In [254]:
#Noticed year missmatch between "year"	"released" columns, on the year value
#I also realised that the year column had the most accurate year so I will 
#First I will convert to another date format and than I will change the year
#FIRST STEP # I was trying to change the format of the date and remove the location 
#however I was getting value of error becasue of the format, So I checked the format above and removed those record
def check_format(date_str):
    try:
        # Trying to parse the date on the date_str we will receive
        datetime.strptime(date_str.split(' (')[0], "%B %d, %Y")
        return True
    except Exception as e:
        # If any exception occurs, format does not match, so return False
        return False



df['format_correct'] = df['released'].apply(check_format)

# Filter the DataFrame to find rows where the date format is incorrect
incorrect_format_df = df[~df['format_correct']]
# incorrect_format_df
# len(incorrect_format_df)

df = df[df['format_correct']] #Keep on the dataframe only the records with the correct date format, drop the format correct column
df.drop('format_correct', axis=1, inplace=True)



df['released'] = df['released'].apply(
    lambda x: datetime.strptime(x.split(' (')[0], "%B %d, %Y").strftime("%d-%m-%Y") 
)
#Now I update the year on the "released" column 
df['released'] = df.apply(lambda row: f"{row['released'][:-4]}{row['year']}", axis=1)


df


,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,13-06-1980,8.4,927000,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000,46998772,Warner Bros.,146
1,The Blue Lagoon,R,Adventure,1980,02-07-1980,5.8,65000,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4500000,58853106,Columbia Pictures,104
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,20-06-1980,8.7,1200000,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,18000000,538375067,Lucasfilm,124
3,Airplane!,PG,Comedy,1980,02-07-1980,7.7,221000,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3500000,83453539,Paramount Pictures,88
4,Caddyshack,R,Comedy,1980,25-07-1980,7.3,108000,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6000000,39846344,Orion Pictures,98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7648,Bad Boys for Life,R,Action,2020,17-01-2020,6.6,140000,Adil El Arbi,Peter Craig,Will Smith,United States,90000000,426505244,Columbia Pictures,124
7649,Sonic the Hedgehog,PG,Action,2020,14-02-2020,6.5,102000,Jeff Fowler,Pat Casey,Ben Schwartz,United States,85000000,319715683,Paramount Pictures,99
7650,Dolittle,PG,Adventure,2020,17-01-2020,5.6,53000,Stephen Gaghan,Stephen Gaghan,Robert Downey Jr.,United States,175000000,245487753,Universal Pictures,101
7651,The Call of the Wild,PG,Adventure,2020,21-02-2020,6.8,42000,Chris Sanders,Michael Green,Harrison Ford,Canada,135000000,111105497,20th Century Studios,100


In [256]:
# print(df.columns)

# Ordering the records acording to the gross value

df.sort_values(by=['gross'], inplace=False, ascending=False)

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
5445,Avatar,PG-13,Action,2009,18-12-2009,7.8,1100000,James Cameron,James Cameron,Sam Worthington,United States,237000000,2847246203,Twentieth Century Fox,162
7445,Avengers: Endgame,PG-13,Action,2019,26-04-2019,8.4,903000,Anthony Russo,Christopher Markus,Robert Downey Jr.,United States,356000000,2797501328,Marvel Studios,181
3045,Titanic,PG-13,Drama,1997,19-12-1997,7.8,1100000,James Cameron,James Cameron,Leonardo DiCaprio,United States,200000000,2201647264,Twentieth Century Fox,194
6663,Star Wars: Episode VII - The Force Awakens,PG-13,Action,2015,18-12-2015,7.8,876000,J.J. Abrams,Lawrence Kasdan,Daisy Ridley,United States,245000000,2069521700,Lucasfilm,138
7244,Avengers: Infinity War,PG-13,Action,2018,27-04-2018,8.4,897000,Anthony Russo,Christopher Markus,Robert Downey Jr.,United States,321000000,2048359754,Marvel Studios,149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5640,Tanner Hall,R,Drama,2009,15-01-2009,5.8,3500,Francesca Gregorini,Tatiana von Fürstenberg,Rooney Mara,United States,3000000,5073,Two Prong Lesson,96
2434,Philadelphia Experiment II,PG-13,Action,1993,04-06-1993,4.5,1900,Stephen Cornwell,Wallace C. Bennett,Brad Johnson,United States,5000000,2970,Trimark Pictures,97
3681,Ginger Snaps,Not Rated,Drama,2000,11-05-2000,6.8,43000,John Fawcett,Karen Walton,Emily Perkins,Canada,5000000,2554,Copperheart Entertainment,108
272,Parasite,R,Horror,1982,12-03-1982,3.9,2300,Charles Band,Alan J. Adler,Robert Glaudini,United States,800000,2270,Embassy Pictures,85


In [1]:
# pd.set_option("display.max_rows", None) #If we want to dispaly the whole dataset

In [2]:
#Checking and drop  duplicates
df[]